<p>
<img src="../imgs/EII-ULPGC-logo.jpeg" width="430px" align="right">

# **NOTEBOOK 17**
---



# **Modelos del lenguaje basados en redes neuronales artificiales**

## **Transformers**

https://jalammar.github.io/illustrated-transformer/

https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html#transformer_intro


Los modelos Transformer son una clase de arquitecturas de aprendizaje profundo que han revolucionado el campo del procesamiento del lenguaje natural (NLP) y más allá. Introducidos en el paper "Attention Is All You Need" por Vaswani et al. en 2017, los Transformers se destacan por su capacidad para manejar secuencias de datos, como texto, de una manera muy eficiente y efectiva. A diferencia de las arquitecturas anteriores como las redes neuronales recurrentes (RNN) y las redes neuronales convolucionales (CNN), los Transformers se basan principalmente en un mecanismo llamado "atención", especialmente la "atención de múltiples cabezas".

<p align="center">
<img src="imgs/Transformer.png" width="30%">
</p>

Aquí hay algunos puntos clave sobre los modelos Transformer:

1. **Mecanismo de Atención**: El componente central de un Transformer es el mecanismo de atención, que permite al modelo ponderar la importancia de diferentes partes de la entrada. En el contexto del NLP, esto significa que un Transformer puede prestar más atención a palabras relevantes y menos a las irrelevantes al procesar un texto.

2. **Atención de Múltiples Cabezas**: Los Transformers utilizan lo que se llama "atención de múltiples cabezas". Esto les permite prestar atención a diferentes partes de la secuencia de entrada simultáneamente, lo que mejora la capacidad del modelo para aprender relaciones complejas.

3. **Arquitectura de Codificador y Decodificador**: Los Transformers originales se componen de una serie de bloques de codificadores y decodificadores. El codificador procesa la entrada (por ejemplo, un texto en un idioma fuente para la traducción automática), y el decodificador genera la salida (por ejemplo, el texto traducido). Cada bloque de codificador y decodificador contiene capas de atención y redes neuronales feed-forward.

4. **Paralelización y Escalabilidad**: A diferencia de las RNN, los Transformers no requieren que los datos se procesen secuencialmente. Esto significa que pueden manejar secuencias completas de datos a la vez, lo que permite una paralelización masiva y hace que los Transformers sean particularmente adecuados para el hardware moderno de GPU.

5. **Aplicaciones en NLP y Más Allá**: Aunque los Transformers fueron diseñados originalmente para tareas de NLP como la traducción automática, su aplicación se ha extendido a una amplia gama de tareas, incluyendo la generación de texto (como GPT-3), el entendimiento del lenguaje (como BERT), el procesamiento de imágenes, y más.

6. **Transferencia y Aprendizaje Pre-entrenado**: Modelos como BERT y GPT-3 son pre-entrenados en grandes cantidades de datos y luego afinados en tareas específicas, lo que les permite alcanzar un rendimiento destacado en una amplia gama de tareas de NLP con relativamente poco esfuerzo de ajuste específico de la tarea.




## **Mecanismo de auto-atención del modelo Transformer**

El mecanismo de autoatención con enmascaramiento en el Transformer permite que cada token en una secuencia determine cuánto debe “atender” a otros tokens en esa secuencia. La imagen siguiente ilustra este proceso paso a paso. A continuación, se describe cada etapa del mecanismo de autoatención con enmascaramiento:

#### **Paso 1: Cálculo de las puntuaciones de atención**

**Matrices de consultas (Q), claves (K) y valores (V):**

- Cada token en la secuencia se representa mediante un vector, y se generan tres matrices para cada token: Q (consultas), K (claves) y V (valores). En la imagen, tenemos cuatro tokens: q1, q2, q3, q4 en Q y k1, k2, k3, k4 en K.
- Estas matrices se utilizan para determinar la relevancia entre los tokens en la secuencia.

**Multiplicación de matrices:**
- La matriz de consultas (Q) se multiplica con la matriz de claves (K) para obtener una matriz de puntuaciones de atención (scores).
- Cada elemento de la matriz de scores representa la relevancia entre un par de tokens: por ejemplo, el valor en la posición (q1, k2) representa la importancia de k2 respecto a q1.

#### **Paso 2: Escalado de las puntuaciones de atención**

- Para estabilizar los valores, se divide cada puntuación por la raíz cuadrada de la dimensión de los vectores K (normalmente denotada como $\sqrt{d_k}$). Esto evita que los valores sean excesivamente grandes, lo cual podría dificultar la convergencia del modelo.

#### **Paso 3: Aplicación de la matriz de enmascaramiento**

**Matriz de enmascaramiento:**

- En modelos como GPT, se utiliza una matriz de enmascaramiento causal para garantizar que cada token solo pueda “ver” los tokens anteriores (incluyendo el actual), evitando que el modelo acceda a tokens futuros.
- En la imagen, vemos una matriz de enmascaramiento triangular que enmascara los valores superiores de la matriz de scores, donde solo se permiten los valores de la diagonal y los elementos inferiores. Los valores enmascarados se establecen en cero para evitar que el modelo los considere.

**Aplicación de la máscara:**
- La matriz de enmascaramiento se multiplica con la matriz de puntuaciones escaladas. Los valores enmascarados se convierten en cero, indicando que esos tokens no deben ser tenidos en cuenta en el cálculo de atención.

#### **Paso 4: Aplicación de la función Softmax**

- Después de enmascarar los valores, se aplica la función softmax en cada fila de la matriz resultante.
- La función softmax convierte cada fila en una distribución de probabilidad, resaltando los tokens más relevantes en la atención y atenuando los tokens menos relevantes.
- Esto permite que cada token asigne mayor “atención” a los tokens relevantes en función de su contexto en la secuencia.

#### **Paso 5: Multiplicación con la matriz de valores (V)**

**Cálculo del vector de salida:**
- La matriz de probabilidades obtenida después de la softmax se multiplica con la matriz de valores (V) para obtener el vector de atención final para cada token.
- En la imagen, los vectores resultantes z1, z2, z3, z4 corresponden a los valores ponderados de cada token según su relevancia.

**Resultado final:**
- Los vectores z1, z2, z3, z4 representan la salida de la capa de atención para cada token. Estos vectores contienen información agregada sobre los tokens anteriores en la secuencia, ajustada según la relevancia calculada.

<div align="center">
<img src="./imgs/Attention_QKV_mask.svg" width="70%">
</div>  


---

### **Práctica: implementación del mecanismo de auto-atención con enmascaramiento del modelo Transformer**

Vamos a implementar el mecanismo de auto-atención con enmascaramiento del modelo Transformer en Pytorch. Para ello, vamos a seguir los pasos descritos anteriormente y suponer que ya tenemos las matrices de consultas (Q), claves (K) y valores (V) para cada token en la secuencia.


In [1]:
import torch

Q = torch.tensor([[0.0, 0.0, 0.0], [1, 1, 1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3]])
K = torch.tensor([[0.1, 0.1, 0.1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3], [0.4, 0.4, 0.4]])
V = torch.tensor([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.], [0., 1., 1.]])

El enmascaramiento durante la etapa del decodificador en los modelos Transformer es crucial para evitar que el decodificador tenga acceso a información futura, especialmente en tareas de generación secuencial como la traducción automática o la generación de texto. Este concepto se conoce como "enmascaramiento de atención causal".

En el contexto de los Transformers, el decodificador genera una salida secuencialmente, palabra por palabra. Durante la generación de cada palabra, es importante que el modelo solo tenga en cuenta las palabras anteriores y no las futuras, ya que estas últimas no deberían estar disponibles (en un escenario de generación de texto, por ejemplo, las palabras futuras aún no se han generado).

Una vez realizado el resultado debe ser:

<table>
<tr>
<td><b>z1</b></td><td>1.0000</td><td>0.0000</td><td>0.0000</td>
</tr>
<tr>
<td><b>z2</b></td><td>0.4568</td><td>0.5432</td><td>0.0000</td>
</tr>
<tr>
<td><b>z3</b></td><td>0.3219</td><td>0.3332</td><td>0.3449</td>
</tr>
<tr>
<td><b>z4</b></td><td>0.2309</td><td>0.5130</td><td>0.5260</td>
</tr>
</table>

#### **Objetivos de la práctica**

- Entender con detalle el funcionamiento del mecanismo de auto-atención con enmascaramiento.
- Practicar las operaciones matriciales en PyTorch.

---

## **Layer Normalization**


"Layer Normalization" es una técnica de normalización utilizada en el entrenamiento de redes neuronales profundas, introducida por Jimmy Lei Ba, Jamie Ryan Kiros y Geoffrey Hinton en 2016 (https://arxiv.org/pdf/1607.06450.pdf). Esta técnica es fundamental para mejorar la estabilidad y acelerar el entrenamiento de modelos de aprendizaje profundo, especialmente en el contexto de redes neuronales recurrentes (RNN) y modelos de atención como los Transformers.

### Conceptos clave de la Layer Normalization:

1. **Normalización por Ejemplo**: La Layer Normalization normaliza los datos a lo largo de las características para cada ejemplo individual en un lote. Esto significa que, para cada dato en el lote, la normalización se realiza calculando la media y la varianza de todas sus características.

2. **Cálculo de la Media y la Varianza**: Para cada ejemplo, se calcula la media y la varianza de todas sus características. Estos valores se utilizan para normalizar las características de ese ejemplo específico.

3. **Proceso de Normalización**: La normalización se realiza restando la media y dividiendo por la desviación estándar (raíz cuadrada de la varianza) de las características de cada ejemplo. Se añade un pequeño valor, conocido como epsilon, al denominador para evitar la división por cero.

4. **Parámetros Aprendibles**: Después de normalizar las características, se aplican dos parámetros aprendibles a cada una de ellas: un factor de escala (gamma) y un término de desplazamiento (beta). Estos parámetros son específicos para cada capa de la red y se ajustan durante el proceso de entrenamiento.

5. **Aplicaciones en Modelos de Secuencia**: La Layer Normalization ha demostrado ser particularmente efectiva en modelos que procesan datos secuenciales, como las redes neuronales recurrentes y los modelos basados en el mecanismo de atención, donde ayuda a mejorar la estabilidad y eficiencia del entrenamiento.

### Ventajas de la Layer Normalization:

- **Estabilidad en el Entrenamiento**: Contribuye a la estabilidad del entrenamiento de redes neuronales profundas, lo que es crucial para lograr un buen rendimiento en tareas complejas.
- **Eficiencia en Modelos de Secuencia**: Facilita el entrenamiento eficiente de modelos que trabajan con datos secuenciales, mejorando tanto la velocidad de convergencia como la calidad del modelo entrenado.
- **Flexibilidad en el Tamaño del Lote**: Al normalizar cada ejemplo de forma independiente, la Layer Normalization no depende del tamaño del lote, lo que la hace flexible para diferentes configuraciones de entrenamiento.

### Representación matemática:

La Layer Normalization se centra en normalizar las activaciones dentro de una capa para cada ejemplo de datos. A continuación se describe el proceso matemáticamente:

Dado un tensor de entrada $ X $ para una capa específica en la red, donde $ X $ tiene dimensiones $[N, F]$ (con $ N $ siendo el tamaño del lote y $ F $ el número de características o neuronas en la capa), la Layer Normalization se realiza de la siguiente manera para cada ejemplo $ n $ en el lote:

1. **Calcular la Media**:
   $$ \mu_n = \frac{1}{F} \sum_{f=1}^{F} x_{nf} $$
   Aquí, $ \mu_n $ es la media de las activaciones para el ejemplo $ n $.

2. **Calcular la Varianza**:
   $$ \sigma_n^2 = \frac{1}{F} \sum_{f=1}^{F} (x_{nf} - \mu_n)^2 $$
   Donde $ \sigma_n^2 $ es la varianza de las activaciones para el ejemplo $ n $.

3. **Normalizar**:
   $$ \hat{x}_{nf} = \frac{x_{nf} - \mu_n}{\sqrt{\sigma_n^2 + \epsilon}} $$
   Cada activación $ x_{nf} $ se normaliza restando la media $ \mu_n $ y dividiendo por la raíz cuadrada de la varianza $ \sigma_n^2 $, con un pequeño número $ \epsilon $ añadido para la estabilidad numérica (evitar la división por cero).

4. **Aplicar Parámetros Aprendibles**:
   $$ y_{nf} = \gamma \hat{x}_{nf} + \beta $$
   Finalmente, se aplica una transformación lineal a las activaciones normalizadas, donde $ \gamma $ y $ \beta $ son parámetros aprendibles específicos de la capa. Estos parámetros permiten que la normalización se ajuste de manera flexible durante el entrenamiento.

En esta formulación, $ n $ indexa los ejemplos en el lote y $ f $ indexa las características. Cada ejemplo se normaliza de forma independiente. Los parámetros $ \gamma $ y $ \beta $ son de la misma dimensión que el número de características $ F $, permitiendo una escala y un desplazamiento distintos para cada característica.

La Layer Normalization es especialmente útil en modelos de secuencias como los Transformers y las RNN, donde normaliza las activaciones a lo largo de las características para cada paso de tiempo o posición en la secuencia, contribuyendo a un entrenamiento más estable y eficiente.

---

## Ejercicio: Implementación de la normalización de capa

Asumiento los parámetros $\gamma$ y $\beta$ como 1 y 0 respectivamente, desarrolla un código que normalice los siguientes vectores de características:


In [1]:
import torch

v1 = torch.tensor([1.0, 2.0, 3.0])
v2 = torch.tensor([2.0, 2.0, 2.0])
v3 = torch.tensor([23.0, 0.01, 5.0])

El resultado debe ser:

In [2]:
nl = torch.nn.LayerNorm(3, eps=1e-05, elementwise_affine=True)

print(nl(v1))
print(nl(v2))
print(nl(v3))

tensor([-1.2247,  0.0000,  1.2247], grad_fn=<NativeLayerNormBackward0>)
tensor([0., 0., 0.], grad_fn=<NativeLayerNormBackward0>)
tensor([ 1.3838, -0.9446, -0.4392], grad_fn=<NativeLayerNormBackward0>)
